## word2vec と doc2vec

単語や文章を分散表現（意味が似たような単語や文章を似たようなベクトルとして表現）を取得します。

### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/article/library/scipy/template/template_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/article/library/scipy/template/template_nb.ipynb)

### 筆者の環境
筆者のOSはmacOSです。LinuxやUnixのコマンドとはオプションが異なります。

In [1]:
!sw_vers

ProductName:	Mac OS X
ProductVersion:	10.14.6
BuildVersion:	18G6020


In [2]:
!python -V
!python -V

Python 3.7.3


基本的なライブラリをインポートしそのバージョンを確認しておきます。

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib
import matplotlib.pyplot as plt
import scipy
import numpy as np

print('matplotlib version :', matplotlib.__version__)
print('scipy version :', scipy.__version__)
print('numpy version :', np.__version__)

matplotlib version : 3.0.3
scipy version : 1.4.1
numpy version : 1.19.4


### aa

青空文庫からすべての作品をダウンロード

gitがかなり重いので、最新の履歴だけを取得します。

```bash
git clone --depth 1 https://github.com/aozorabunko/aozorabunko.git
```

In [4]:
!ls -a

.                       .gitignore              aozorabunko             w2v_nb.md               wagahaiwa_nekodearu.txt
..                      .ipynb_checkpoints      w2v_nb.ipynb            w2v_nb.py


実際のファイルはcardsにzip形式として保存されているようです。

In [6]:
ls ./aozorabunko/cards/*. | wc -l

    1118


zipファイルだけzipsに移動させます。

```bash
find ./aozorabunko/cards/ -name *.zip | xargs -I{} cp {} -t ./zips/
```

In [9]:
!ls ./zips/ | head -n 5

1000_ruby_2956.zip
1001_ruby_2229.zip
1002_ruby_20989.zip
1003_ruby_2008.zip
1004_ruby_2053.zip


In [10]:
!ls ./zips/ | wc -l

   16442


となり、

```bash
for i in `ls`; do [[ ${i##*.} == zip ]] && unzip -o $i -d ../texts/; done
```

In [90]:
import re

with open('wagahaiwa_nekodearu.txt', mode='r') as f:
  all_sentence = f.read()

all_sentence = all_sentence.replace(" ", "").replace("　","").replace("\n","").replace("|","")
all_sentence = re.sub("《[^》]+》", "", all_sentence)

sentence_list = all_sentence.split("。")

sentence_list = [ s + "。" for s in sentence_list]

sentence_list[:10]

['吾輩は猫である。',
 '名前はまだ無い。',
 'どこで生れたかとんと見当がつかぬ。',
 '何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。',
 '吾輩はここで始めて人間というものを見た。',
 'しかもあとで聞くとそれは書生という人間中で一番｜獰悪な種族であったそうだ。',
 'この書生というのは時々我々を捕えて煮て食うという話である。',
 'しかしその当時は何という考もなかったから別段恐しいとも思わなかった。',
 'ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。',
 '掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。']

### janomeによる形態素解析

In [91]:
from janome.tokenizer import Tokenizer

t = Tokenizer()

word_list = []
word_per_sentence_list = []
for sentence in sentence_list:
  word_list.extend(list(t.tokenize(sentence, wakati=True)))
  word_per_sentence_list.append(list(t.tokenize(sentence, wakati=True)))
  

print(word_list[0])
print(word_per_sentence_list[0])

### 単語のカウント

単語のカウントを行い、出現頻度の高いベスト10を抽出してみます。

In [73]:
import collections

count = collections.Counter(word_list)
count.most_common()[:10]

[('。', 14976),
 ('の', 9226),
 ('て', 6868),
 ('、', 6806),
 ('に', 6569),
 ('は', 6458),
 ('を', 6071),
 ('と', 5512),
 ('が', 5336),
 ('た', 3991)]

### gensimに含まれるword2vecを用いた学習

word2vecを用いて、word_listの分散表現を取得します。

In [74]:
from gensim.models import word2vec

model = word2vec.Word2Vec(word_list, size=100, min_count=5, window=5, iter=20, sg=0)

#### 分散行列

In [75]:
model.wv.vectors

array([[-4.6434267e-03, -2.3294671e-03, -4.6703969e-03, ...,
        -1.5287000e-03,  2.5717122e-03, -3.9563971e-03],
       [-4.8714927e-01,  5.5891466e-01, -6.4149547e-01, ...,
        -2.4196199e-01,  1.5717342e-01, -4.4097636e-02],
       [ 4.8585821e-02,  7.7471709e-01, -5.7659220e-02, ...,
        -1.8813603e-02,  7.9032749e-01, -9.8730542e-02],
       ...,
       [ 1.6272579e-01, -3.3262840e-01,  3.7692271e-02, ...,
         6.5525115e-02, -1.7334071e-01, -1.1514240e-01],
       [-2.7475036e-03, -1.9515246e-04,  8.7064004e-04, ...,
        -1.3576463e-04,  5.0814112e-04, -3.6014980e-03],
       [ 7.4753255e-02, -1.8811050e-01, -2.1342296e-02, ...,
         4.8131797e-02,  9.1184732e-03, -8.6615857e-04]], dtype=float32)

#### 分散行列の形状確認

In [76]:
model.wv.vectors.shape

(1944, 100)

In [77]:
model.wv.index2word[:10]

['。', 'の', 'い', 'る', 'て', 'な', 'と', 'に', 'か', 'し']

In [78]:
model.wv.vectors[0]

array([-4.6434267e-03, -2.3294671e-03, -4.6703969e-03,  3.2870518e-03,
        3.9587654e-03,  2.7448912e-03,  5.1382475e-04,  1.4591415e-03,
        3.4210815e-03,  4.6661599e-03, -2.9773691e-03,  3.1596429e-03,
        8.1559032e-04, -2.5696231e-03,  4.6706498e-03,  2.9519293e-03,
        1.7721216e-03, -4.6404847e-03, -2.1053969e-03,  1.5397997e-03,
        2.9709847e-03, -3.3830637e-03,  2.0975310e-03,  1.8231223e-03,
       -3.9987830e-03,  3.8240411e-04,  4.7334390e-05,  3.4339696e-03,
        2.8869393e-03,  3.8880424e-04, -6.0641230e-04, -4.7422000e-03,
        2.5653825e-03, -4.3430886e-05, -3.1883842e-03, -2.3613521e-03,
       -1.9647183e-03, -2.4281435e-03, -7.8332075e-04,  1.8938017e-03,
        4.5761340e-03,  4.8303121e-04, -2.5999304e-03,  6.5069806e-05,
       -7.3551037e-04,  8.1960758e-04, -1.0901866e-03, -2.8464927e-03,
        1.9212395e-03,  3.3764082e-03,  9.2475110e-04, -3.1201197e-03,
       -2.0914031e-03, -9.5066329e-04,  3.2868241e-03, -4.4226809e-03,
      

In [79]:
model.wv.__getitem__("の")

array([-0.48714927,  0.55891466, -0.64149547,  0.05501885,  1.8577155 ,
        0.1576188 ,  0.2725423 , -0.38784787,  0.16658841, -0.40488723,
       -0.49220562, -0.8612247 , -0.7580111 , -0.65297765,  0.38963282,
       -0.46739623, -0.46161538, -0.81573534, -0.08132691,  0.09076468,
        0.01756369,  0.18738756,  0.57488316, -0.3031986 ,  0.3349143 ,
       -0.29604337, -0.6802442 , -0.08843175, -0.31758675,  0.46896583,
        0.62573063, -0.0736936 , -0.49499717,  0.65183896,  0.04701501,
       -0.04857261, -0.19154152, -0.1300338 ,  0.3523873 , -0.5718451 ,
       -0.10662493, -0.20408699, -0.73876035, -0.8125423 , -0.51936895,
        0.25769728,  0.1957122 ,  0.14964166, -0.7879062 , -0.3130662 ,
       -0.01189006, -1.272633  , -0.32655442,  1.001265  , -0.45040557,
        0.7867269 ,  0.37562853,  0.02214631,  0.33811396, -0.04894516,
        0.49913803,  0.31990734, -0.6302795 ,  1.0059496 , -0.6496076 ,
       -0.28672186,  0.1169221 ,  0.00560467,  0.08263827,  0.50

### cos類似度による単語抽出

In [80]:
model.wv.most_similar("男")

[('硝', 0.7872987985610962),
 ('厨', 0.7846057415008545),
 ('富', 0.7842417359352112),
 ('玉', 0.7794391512870789),
 ('菓', 0.7785281538963318),
 ('童', 0.7761118412017822),
 ('孔', 0.7702925205230713),
 ('帽', 0.7676213979721069),
 ('辟', 0.7570321559906006),
 ('虚', 0.7524005174636841)]

### 単語ベクトルによる演算

足し算するにはpositiveメソッドを引き算にはnegativeメソッドを利用します。

まず、猫＋人を計算します。

In [81]:
model.wv.most_similar(positive=["猫", "人"])

[('胴', 0.7437920570373535),
 ('傚', 0.7369073033332825),
 ('朋', 0.7276594638824463),
 ('週', 0.7174804210662842),
 ('隙', 0.7084348201751709),
 ('夫', 0.7067784070968628),
 ('欄', 0.7014670372009277),
 ('接', 0.683977484703064),
 ('板', 0.678753137588501),
 ('茶', 0.6730451583862305)]

次に猫＋人ー男を計算します。

In [82]:
model.wv.most_similar(positive=["猫", "人"], negative=["男"])

[('茶', 0.6552659869194031),
 ('傚', 0.6030101776123047),
 ('互', 0.5300031304359436),
 ('週', 0.5223612189292908),
 ('接', 0.5223289728164673),
 ('中', 0.5197547078132629),
 ('客', 0.517535388469696),
 ('義', 0.5164977312088013),
 ('世', 0.5087952017784119),
 ('多', 0.5071096420288086)]

## doc2vec

文章毎にタグ付けされたTaggedDocumentを作成します。

In [92]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

tagged_doc_list = []

for i, sentence in enumerate(word_per_sentence_list):
  tagged_doc_list.append(TaggedDocument(sentence, [i]))

print(tagged_doc_list[0])

TaggedDocument(['吾輩', 'は', '猫', 'で', 'ある', '。'], [0])


In [93]:
model = Doc2Vec(documents=tagged_doc_list, vector_size=100, min_count=5, window=5, epochs=20, dm=0)

In [94]:
word_per_sentence_list[0]

['吾輩', 'は', '猫', 'で', 'ある', '。']

In [95]:
model.docvecs[0]

array([-0.08167019, -0.12457962,  0.12471442, -0.09050006, -0.03635857,
        0.04212522, -0.01509753,  0.00351651,  0.05914155, -0.00873887,
        0.01239055, -0.02777859,  0.16134271, -0.01632613, -0.10115265,
        0.184944  , -0.10290116, -0.0909429 ,  0.02121286,  0.05977118,
       -0.01025303, -0.02403504,  0.07107952, -0.05833486, -0.0182417 ,
        0.05880298,  0.18948261, -0.02985778,  0.13717374,  0.06628802,
       -0.05900386,  0.0487904 ,  0.08658132, -0.08866487,  0.12545186,
        0.03065981, -0.01357306, -0.11977363,  0.05700683,  0.00365328,
        0.02353869,  0.00135398, -0.06402358, -0.10522079,  0.04675397,
       -0.14536764, -0.17137018, -0.03801022, -0.09044138,  0.21002217,
       -0.04263447, -0.0945474 ,  0.03820987,  0.10682409, -0.09301145,
       -0.14245296, -0.0528282 , -0.06055163, -0.01451581,  0.0213145 ,
       -0.03098489, -0.04248807, -0.01198582,  0.1051307 ,  0.05513797,
       -0.04387281, -0.05777524,  0.0328118 , -0.00943001, -0.01

most_similarで類似度が高い文章のIDと類似度を取得することが出来ます。

In [96]:
model.docvecs.most_similar(0)

[(157, 0.9983920454978943),
 (425, 0.9979491829872131),
 (3783, 0.9979193210601807),
 (79, 0.9978989362716675),
 (4582, 0.9978306293487549),
 (1757, 0.9978129863739014),
 (5138, 0.9977301955223083),
 (5886, 0.9977244138717651),
 (2430, 0.9976636171340942),
 (3605, 0.9976422786712646)]

In [98]:
for p in model.docvecs.most_similar(0):
  print(word_per_sentence_list[p[0]])

['吾輩', 'は', '猫', 'として', '決して', '上乗', 'の', '出来', 'で', 'は', 'ない', '。']
['吾輩', 'も', 'この', '頃', 'で', 'は', '普通', '一般', 'の', '猫', 'で', 'は', 'ない', '。']
['吾輩', 'は', '淡泊', 'を', '愛する', '茶人', '的', '猫', 'で', 'ある', '。']
['吾輩', 'は', '猫', 'ながら', '時々', '考える', '事', 'が', 'ある', '。']
['今', '吾輩', 'が', '記述', 'する', 'ベースボール', 'は', 'この', '特別', 'の', '場合', 'に', '限ら', 'るる', 'ベースボール', '即ち', '攻', '城', '的', '砲術', 'で', 'ある', '。']
['ことに', '吾輩', 'は', 'この', '道', 'に', '掛け', 'て', 'は', '日本一', 'の', '堪能', 'で', 'ある', '。']
['吾輩', 'ひそか', 'に', '思う', 'に', 'この', '状態', 'は', '決して', '胎毒', 'や', '疱瘡', 'の', 'ため', 'で', 'は', 'ない', '。']
['吾輩', 'も', '日本', 'の', '猫', 'だ', 'から', '多少', 'の', '愛国心', 'は', 'ある', '。']
['吾輩', '目下', 'の', '状態', 'は', 'ただ', '休養', 'を', '欲する', 'のみ', 'で', 'ある', '。']
['吾輩', 'の', '取る', 'の', 'は', 'この', '蟻', 'の', '領分', 'に', '寝転ん', 'で', 'いる', '奴', 'で', 'は', 'ない', '。']


感覚的ですが、似たような文章が抽出されています。